Perfect. Let’s keep everything **inside your new modeling notebook**—no refactor. Below is a tight, copy-paste sequence of cells you can run top-to-bottom. It gives you: clean features, **chrono split** to pick a model quickly, **rolling backtest** to verify robustness, and side-by-side metrics/plots. Minimal moving parts; all per-province; leak-safe.

---

# Cell 1 — Imports & assumptions

```python
import pandas as pd, numpy as np
from typing import Iterator, Tuple, Callable, Dict
import matplotlib.pyplot as plt

# Assumes df has columns: 'province', 'date' (datetime64), 'starts' (target)
# and is already sorted by ['province','date'].
assert set(['province','date','starts']).issubset(df.columns)
df = df.copy()
df['province'] = df['province'].astype('category')
```

**Intuition:** lock types now; prevents groupby surprises later.

---

# Cell 2 — Lightweight features (leak-safe, per province)

```python
def add_feats_per_panel(df, target='starts', lags=(1,2,3,6,12), diffs=(1,12)):
    def _f(g):
        g = g.copy()
        g['month'] = g['date'].dt.month
        for L in lags:  g[f'{target}_lag{L}'] = g[target].shift(L)
        for d in diffs: g[f'{target}_diff{d}'] = g[target].diff(d)
        return g
    return (df.sort_values(['province','date'])
              .groupby('province', group_keys=False).apply(_f))

dfX = add_feats_per_panel(df, target='starts')
feature_cols = [c for c in dfX.columns if c not in ['province','date','starts']]
```

**Intuition:** all transforms happen **within** each province → no cross-panel leakage.

---

# Cell 3 — Splitters (chrono + rolling origin)

```python
def chrono_split(df, cutoff):
    tr = df[df['date'] <= cutoff].copy()
    te = df[df['date'] >  cutoff].copy()
    return tr, te

def rolling_origin(df, initial=None, step=1, fh=3) -> Iterator[Tuple[pd.DataFrame,pd.DataFrame]]:
    dates = np.sort(df['date'].unique())
    if initial is None: initial = dates[int(0.6*len(dates))]
    start_i = np.where(dates==initial)[0][0]
    for i in range(start_i, len(dates)-fh, step):
        train_end = dates[i]
        test_chunk = dates[i+1:i+1+fh]
        yield df[df['date'] <= train_end].copy(), df[df['date'].isin(test_chunk)].copy()
```

**Intuition:** choose model fast with **chrono**; confirm stability with **rolling**.

---

# Cell 4 — Baseline (seasonal naive) + metrics

```python
def seasonal_naive(hist_series, season=12):
    return hist_series.shift(season)

def seasonal_naive_predict(train, test, target='starts', season=12):
    preds = []
    for prov, gte in test.groupby('province'):
        gtr = train[train['province']==prov]
        hist_plus_future = pd.concat([gtr, gte]).sort_values('date')
        yhat = seasonal_naive(hist_plus_future[target], season).loc[gte.index]
        preds.append(yhat)
    return pd.concat(preds).sort_index()

def smape(y, yhat):
    y, yhat = y.astype(float), yhat.astype(float)
    denom = (np.abs(y)+np.abs(yhat))
    denom = np.where(denom==0, 1.0, denom)
    return 200*np.mean(np.abs(y - yhat)/denom)

def mase(y, yhat, insample, season=12):
    denom = np.abs(insample[season:] - insample[:-season]).mean()
    return np.mean(np.abs(y - yhat)) / (denom if denom!=0 else np.nan)

def metric_table(test_df, yhat, target='starts'):
    out = test_df[['province','date',target]].copy()
    out['yhat'] = yhat.values
    out['h'] = out.groupby('province')['date'].rank(method='first').astype(int)
    g = out.groupby('h')
    return pd.DataFrame({
        'RMSE': g.apply(lambda s: np.sqrt(np.mean((s[target]-s['yhat'])**2))),
        'sMAPE': g.apply(lambda s: smape(s[target], s['yhat']))
    }).reset_index()
```

**Intuition:** this baseline is your bar. If you can’t beat it, stop.

---

# Cell 5 — Quick chrono check (pick a sensible model)

```python
cutoff = pd.Timestamp('2023-12-01')  # adjust if needed
train, test = chrono_split(dfX, cutoff=cutoff)

yhat_naive = seasonal_naive_predict(train, test, 'starts', season=12)
mtx_naive = metric_table(test, yhat_naive, 'starts')
mtx_naive
```

**Intuition:** instant reality check per horizon.

---

# Cell 6 — SARIMAX (statsmodels) for seasonality; ETS as backup

```python
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.tsa.holtwinters import ExponentialSmoothing

def fit_sarimax_panel(g, target='starts', order=(1,1,1), seasonal_order=(0,1,1,12)):
    y = g[target]
    m = SARIMAX(y, order=order, seasonal_order=seasonal_order,
                enforce_stationarity=False, enforce_invertibility=False)
    return m.fit(disp=False)

def sarimax_predict_panel(res, steps):
    return res.get_forecast(steps=steps).predicted_mean

def fit_ets_panel(g, target='starts', m=12):
    y = g[target]
    m = ExponentialSmoothing(y, trend='add', seasonal='add', seasonal_periods=m)
    return m.fit()

def ets_predict_panel(res, steps):
    return res.forecast(steps)
```

**Intuition:** SARIMAX handles calendar seasonality; ETS is a strong low-effort alternative.

---

# Cell 7 — ML with lagged features (Ridge + GBR)

```python
from sklearn.linear_model import Ridge
from sklearn.ensemble import GradientBoostingRegressor

def prepare_supervised(g, target='starts', cols=None):
    cols = cols or [c for c in g.columns if c not in ['province','date',target]]
    mask = g[cols].notna().all(axis=1)
    return g.loc[mask, cols], g.loc[mask, target], mask

def fit_ridge(g, target='starts', cols=None, alpha=1.0):
    X, y, mask = prepare_supervised(g, target, cols)
    model = Ridge(alpha=alpha).fit(X, y)
    return model, cols

def fit_gbr(g, target='starts', cols=None, params=None):
    X, y, mask = prepare_supervised(g, target, cols)
    model = GradientBoostingRegressor(**(params or {'n_estimators':400,'max_depth':3})).fit(X, y)
    return model, cols

def predict_model(model, cols, g_future):
    Xf, _, maskf = prepare_supervised(g_future, cols=cols)
    yhat = pd.Series(model.predict(Xf), index=g_future.index[maskf])
    # reindex to future (may have NaNs if first rows lack lags)
    return yhat.reindex(g_future.index)
```

**Intuition:** use your lag features; trees/linear often beat classical models when nonlinearities exist.

---

# Cell 8 — A tiny backtest harness (works for SARIMAX/ETS/ML)

```python
def backtest(df, splitter, target, model_kind='sarimax', **kwargs):
    """
    model_kind in {'sarimax','ets','ridge','gbr','naive'}
    kwargs: pass model hyperparams (order, seasonal_order, cols, etc.)
    """
    recs = []
    for tr, te in splitter:
        for prov, gtr in tr.groupby('province'):
            gte = te[te['province']==prov]
            if gte.empty: continue

            if model_kind=='naive':
                yhat = seasonal_naive_predict(gtr, gte, target, season=kwargs.get('season',12))
            elif model_kind=='sarimax':
                res = fit_sarimax_panel(gtr, target, kwargs.get('order',(1,1,1)), kwargs.get('seasonal_order',(0,1,1,12)))
                yhat = sarimax_predict_panel(res, steps=len(gte)).rename(index=gte.index)
            elif model_kind=='ets':
                res = fit_ets_panel(gtr, target, m=kwargs.get('m',12))
                yhat = ets_predict_panel(res, steps=len(gte)).rename(index=gte.index)
            elif model_kind in {'ridge','gbr'}:
                cols = kwargs.get('cols')
                if model_kind=='ridge':
                    model, cols = fit_ridge(gtr, target, cols, alpha=kwargs.get('alpha',1.0))
                else:
                    model, cols = fit_gbr(gtr, target, cols, params=kwargs.get('params'))
                yhat = predict_model(model, cols, gte)
            else:
                raise ValueError("Unknown model_kind")

            tmp = gte[['province','date',target]].copy()
            tmp['yhat'] = yhat.values
            tmp['h'] = tmp.groupby('province')['date'].rank(method='first').astype(int)
            recs.append(tmp)
    return pd.concat(recs).reset_index(drop=True)
```

**Intuition:** one function, all models; keeps evaluation comparable.

---

# Cell 9 — Run chrono first; choose 1–2 candidates

```python
# Baseline
mtx_naive = metric_table(test, seasonal_naive_predict(train, test, 'starts', 12))
print("Naive sMAPE by horizon:\n", mtx_naive)

# SARIMAX on chrono
bt_sarima_chrono = backtest(dfX, splitter=[(train, test)], target='starts',
                            model_kind='sarimax', order=(1,1,1), seasonal_order=(0,1,1,12))
mtx_sarima = metric_table(bt_sarima_chrono, bt_sarima_chrono['yhat'])
print("SARIMAX sMAPE by horizon:\n", mtx_sarima)

# ML (GBR) on chrono
laggy = [c for c in feature_cols if 'lag' in c or c in ['month']]
bt_gbr_chrono = backtest(dfX, splitter=[(train, test)], target='starts',
                         model_kind='gbr', cols=laggy, params={'n_estimators':500,'max_depth':3})
mtx_gbr = metric_table(bt_gbr_chrono, bt_gbr_chrono['yhat'])
print("GBR sMAPE by horizon:\n", mtx_gbr)
```

**Intuition:** pick the winner(s) that beat seasonal naive on most horizons.

---

# Cell 10 — Rolling backtest on the winner

```python
ro = rolling_origin(dfX, initial=None, step=1, fh=3)

# example: run SARIMAX and GBR
bt_sarima_roll = backtest(dfX, splitter=ro, target='starts',
                          model_kind='sarimax', order=(1,1,1), seasonal_order=(0,1,1,12))
bt_gbr_roll = backtest(dfX, splitter=rolling_origin(dfX, fh=3), target='starts',
                       model_kind='gbr', cols=laggy, params={'n_estimators':500,'max_depth':3})

mtx_sarima_roll = metric_table(bt_sarima_roll, bt_sarima_roll['yhat'])
mtx_gbr_roll    = metric_table(bt_gbr_roll,    bt_gbr_roll['yhat'])

display(mtx_sarima_roll.assign(model='SARIMAX'))
display(mtx_gbr_roll.assign(model='GBR'))
```

**Intuition:** checks stability across start dates; avoids “lucky split” bias.

---

# Cell 11 — Quick plots (per province & horizon)

```python
def plot_last_n(prov='on', n=36, frame=bt_sarima_roll, target='starts'):
    g = frame[frame['province']==prov].sort_values('date').tail(n)
    plt.figure(figsize=(9,3.5))
    plt.plot(g['date'], g[target], label='actual')
    plt.plot(g['date'], g['yhat'], label='forecast')
    plt.title(f'{prov.upper()} — last {n} months'); plt.legend(); plt.tight_layout(); plt.show()

plot_last_n('on', 48, bt_sarima_roll)
plot_last_n('bc', 48, bt_gbr_roll)
```

**Intuition:** eyeball fit; catch systematic lag/overshoot.

---

# Cell 12 — Finalize: refit on full data & export next-k forecasts

```python
def refit_and_forecast_next_k(df, model_kind='sarimax', k=3, target='starts', **kwargs):
    out = []
    for prov, g in df.groupby('province'):
        if model_kind=='sarimax':
            res = fit_sarimax_panel(g, target, kwargs.get('order',(1,1,1)), kwargs.get('seasonal_order',(0,1,1,12)))
            yhat = sarimax_predict_panel(res, steps=k)
        elif model_kind=='ets':
            res = fit_ets_panel(g, target, m=kwargs.get('m',12))
            yhat = ets_predict_panel(res, steps=k)
        elif model_kind in {'ridge','gbr'}:
            cols = kwargs.get('cols')
            model, cols = (fit_ridge(g, target, cols, alpha=kwargs.get('alpha',1.0))
                           if model_kind=='ridge' else
                           fit_gbr(g, target, cols, params=kwargs.get('params')))
            # create k-step “future rows” with months advanced; features must be precomputed externally if needed
            # simplest: use last available lag rows → only 1-step ahead is fully reliable without feature generation
            # Here, we produce 1-step ahead robustly:
            yhat = predict_model(model, cols, g.tail(1))
            yhat = pd.Series([yhat.iloc[-1]]*k, index=pd.RangeIndex(k))  # placeholder if you need exactly k
        else:
            raise ValueError
        out.append(pd.DataFrame({'province':prov, 'h':np.arange(1,k+1), 'yhat':yhat.values}))
    return pd.concat(out, ignore_index=True)

final_sarima = refit_and_forecast_next_k(dfX, model_kind='sarimax', k=3, order=(1,1,1), seasonal_order=(0,1,1,12))
final_sarima.to_csv('forecast_next3_sarimax.csv', index=False)
final_sarima.head()
```

**Intuition:** once validated, train on **all** available history and emit your deliverable.

---

## What to do over the next 7 days (notebook-first)

* **Day 1:** Run Cells 1–5; establish **seasonal naive** bar and pick 1–2 candidate models on chrono split.
* **Day 2–3:** Tune one classical (SARIMAX: `order` and `seasonal_order`) and one ML (GBR: `n_estimators`, `max_depth`) **only if** they already beat the baseline.
* **Day 4:** Run **rolling backtest** (Cell 10). Keep horizon-wise tables; note if any province consistently underperforms.
* **Day 5:** Add **log1p target** variant if variance is large; repeat chrono check quickly. (Wrap `y=log1p(starts)`; invert with `expm1` at the end.)
* **Day 6:** Sanity plots (Cell 11) for 3–4 key provinces; investigate systematic bias (add/remove lags 6/12, add `month`).
* **Day 7:** Refit on full data and **export next-3 forecasts** (Cell 12). Write a short notebook cell summarizing metrics vs baseline and your chosen model.

If you paste these cells now, you can progress without leaving the notebook and still keep rigor (leak-safe, horizon-aware).


## 🧩 Conceptual recap
Step	Already in your notebook?	Purpose

Build leak-free features	✅	Prepare model inputs

Chrono + rolling splits	✅	Respect time order

Baselines	✅	Benchmark performance

Rolling backtest	✅	Cross-validation

Grid search	⚠️ Add manual loop	Tune hyper-parameters

Final refit + forecast	✅	Deploy best model

Exactly ✅ — you don’t need to worry about grid search **right now.**

Here’s the sensible order of operations for you:

---

### 🔹 Step 1 — **Get the whole pipeline working**

* Make sure `build_model_frame`, `chrono_split_q`, and `rolling_origin_q` run cleanly.
* Fit **one baseline** (seasonal naive) and **one model** (e.g., Ridge or SARIMAX).
* Confirm:

  * metrics print correctly (sMAPE, RMSE, etc.),
  * no shape or NaN issues,
  * plots look reasonable.

At this stage, you’re checking that **the plumbing works**, not perfection.

---

### 🔹 Step 2 — **Evaluate models with fixed parameters**

* Use `rolling_origin_q` to backtest Ridge, GBR, or SARIMAX once each.
* Compare their average errors across folds.
* Pick the one that *consistently* beats the baseline.

This already gives you a valid, defendable result for your report.
It’s cross-validated and time-respecting.

---

### 🔹 Step 3 — *(Optional, if time permits later)* Add grid search

Once everything else runs smoothly, drop in that small loop from before to tune parameters.

That’s an *enhancement*, not a missing step.

---

### 🧭 Think of it this way:

* **Now:** build a working, trustworthy forecasting pipeline (end-to-end).
* **Later:** optimize it.

You’ll still get full marks and valid conclusions without a formal grid search —
because your rolling backtest already performs time-series CV, which is the *core* of model validation.
